In [65]:
import secrets
import math

Return GCD of 2 numbers

In [66]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

## Continued Fractions

#### <b>def rational_to_contfrac(p,q)</b>
Converts rational fraction p/q into list of partial quotients [a0, .... , an] <br>

#### <b>def rational_to_contfrac(p,q)</b>
Compute list of convergents using list of partial quotients <br>

#### <b> def contfrac_to_rational(frac) </b>
Converts finite continued fraction [a0, .... , an] to a p/q rational fraction




In [67]:
def rational_to_contfrac(p ,q):
    a = p//q
    quotients = [a]
    while a * q != p:
        p,q = q, p-a*q
        a = p//q
        quotients.append(a)
    return quotients


def convergents_from_contfrac(cf):
    convergents = []
    for i in range(len(cf)):
        convergents.append(contfrac_to_rational(cf[0:i]))
    
    return convergents


def contfrac_to_rational(cf):
    if len(cf) == 0:
        return (0,1)
    
    num = cf[-1]
    denominator = 1 

    for i in range(-2, -len(cf)-1, -1):
        num, denominator = cf[i]*num+denominator, num
    
    return (num, denominator)

Testing the functions

In [68]:
contfrac = rational_to_contfrac(34, 99)
print(contfrac)

conv = convergents_from_contfrac(contfrac)
print(conv)

rational = contfrac_to_rational(contfrac)
print(rational)

[0, 2, 1, 10, 3]
[(0, 1), (0, 1), (1, 2), (1, 3), (11, 32)]
(34, 99)


Rabin-Miller Primality Test <br>
Check if number is prime, p is the number to be tested and s is the number of rounds <br>
S should ensure probabilitiy of p not being prime is < 2^(-80)

In [69]:
def RMPT(p, s):
    q = p - 1
    k = 0 
    
    # if newP % 2 have remainder then stop
    while q % 2 != 1: 
        q //= 2 
        k += 1 

    for i in range(s):
        #Generate random a 
        a = secrets.randbelow(p-2) + 2 

        while gcd(a,p) != 1:
            a = secrets.randbelow(p-2) + 2
            
        z = pow(a, q, p) # z = a^q mod p

        # checking a^q != 1 mod p and a^q != -1 mod p 
        if z != 1 and z != p-1:
            for j in range(1, k):
                z = pow(z, 2, p)
                if z == 1:
                    return False

            if z != p-1:
                return False
    return True

In [94]:
# Check the result 
print(RMPT(58536828032471131348456426324506659436967195846753702250333704608966791528534067798278728747981548972696886762937373961684727824495513261968904996453134494716095278694362125703914706608360435446792042241177272588181082115012537677828367957226772929218486970585153142173891310402614698182537457744036891034121, 2))

False


Generate primes p and q 

In [71]:
def generate_prime(bits, rounds):
    while(True):
        num = secrets.randbits(bits) # using CSRPNG to generate

        # if number is even make it odd 
        if num % 2 == 0:
            num += 1

        elif num % 3 == 0:
            continue 

        elif num % 5 == 0:
            continue

        elif num % 7 == 0:
            continue

        elif num % 11 == 0: 
            continue

        elif RMPT(num, rounds):
            return num


RSA generate weak decryption key using public key e and 2 primes p and q

In [72]:
def RSA_generate_weakd(p, q):
    N = p*q
    d = secrets.randbelow(math.isqrt(math.isqrt(N)) // 3) + 2

    print('N is:', N)
    
    phiN = (p-1) * (q-1)
    gcd_val = gcd(d, phiN)

    while gcd_val != 1:
        d = secrets.randbelow(math.isqrt(math.isqrt(N)) // 3) + 2
        gcd_val = gcd(d, phiN)
    
    print("Weak d = ", d)
    print("Weak d bit length: ", d.bit_length())

    return d


In [73]:
def inv(a, m):
    m0 = m
    x0 = 0
    x1 = 1
    if m == 1:
        return 0
    # Apply extended Euclid Algorithm
    while a > 1:
        q = a // m  # q is quotient

        t = m
        # m is remainder now, process same as euclid's algo
        m = a % m
        a = t

        t = x0

        x0 = x1 - q * x0

        x1 = t

    # Make x1 positive
    if x1 < 0:
        x1 = x1 + m0

    return x1


Encrypt with public key e

In [74]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Decrypt using private key d

In [75]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

In [76]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

In [77]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

## RSA 1024
User of RSA encrypt message using RSA 1024

Generate primes p and q 

User deliberately generate a weak d exponent by setting max bit length to 1/3 * N^1/4 to get faster decryption (< 254 bit)

Encryption exponent e (1024 bit) is obtained by inversing weak d 


In [78]:
phi_test = False

while phi_test == False: 
    p = generate_prime(512, 3)
    q = generate_prime(512, 3)
    
    phiN = (p-1) * (q-1)
    N = p * q

    d = RSA_generate_weakd(p, q)
    e = inv(d, phiN)

    if gcd(e, phiN) == 1:
        phi_test = True
    

    print('E is: ', e)
    print('E bit length: ', e.bit_length())

N is: 92579605280810702884594902014581110314802367855850095291489404277812520295636645313680548969514169470710062755813003007966395224089523930298887206964101605046565530425935697871588079423540061313932281726462133442800015765331755882363060325157598221166234437204910403337156557155378854696979200269273052415329
Weak d =  5680550735152502623822026941641505065132435486688880152081012921638976647109
Weak d bit length:  252
E is:  52251727761912881803247565573838719601058305829641330698939587587657251907877592525346091806261042811105000741834598726301805369586842122135554443815443175172858750480733023028916158559048561516610029262232260025145731758902623713613500525443936838489803152944437624566789042046637540002104572373410658196809
E bit length:  1023


In [79]:
message = "Your Auntie"

ascii_message = convert_to_ascii(message)
ascii_message.bit_length()

encrypted_message = RSA_encrypt(ascii_message, e, N)
print('Encrypted message in ascii: ', encrypted_message)
print('Encrypted message in String: ', convert_to_string(encrypted_message))



Encrypted message in ascii:  18422241215465115228822492614586374810010444399357183692225533736592225052057109728628966914097136231257473989601838285832117585686581614049548205249479173326621682857306575941244234543455082508024146938166386251255557543325132374608822629188540881649736562517639144973357893723681811990694080379989889306568
Encrypted message in String:  Ʀñ×Ǒsä̶ǬɦɊŶ̪
ƼƏť·ʴáȕˠɐá49m˘ɴφΒaçāǙϝə͆ĝ̀uɉʮɅɦ1ȤÍùǟ­ņɭʪ͙ĲȿέôêȟǇRǼΪ¦ƂûÿȭȟŅŶɠ̶ɵ¼ȜͱʉˠȲȅɿύťͽ˓ʩ̫ϞʶPŻϝ͹Ĳȸ


In [80]:
decrypted_message = RSA_decrypt(encrypted_message, d, N)

print('Decrypted msg in ascii: ', decrypted_message)
print('Decrypted msg in string: ', convert_to_string(decrypted_message))

Decrypted msg in ascii:  89111117114032065117110116105101
Decrypted msg in string:  Your Auntie


In [81]:
def bitlength(x):
    '''
    Calculates the bitlength of x
    '''
    assert x >= 0
    n = 0
    while x > 0:
        n = n+1
        x = x>>1
    return n

In [82]:
def isqrt(n):
    '''
    Calculates the integer square root
    for arbitrary large nonnegative integers
    '''
    if n < 0:
        raise ValueError('square root not defined for negative numbers')
    
    if n == 0:
        return 0
    a, b = divmod(bitlength(n), 2)
    x = 2**(a+b)
    while True:
        y = (x + n//x)//2
        if y >= x:
            return x
        x = y

In [83]:
def is_perfect_square(n):
    '''
    If n is a perfect square it returns sqrt(n),
    
    otherwise returns -1
    '''
    h = n & 0xF; #last hexadecimal "digit"
    
    if h > 9:
        return -1 # return immediately in 6 cases out of 16.

    # Take advantage of Boolean short-circuit evaluation
    if ( h != 2 and h != 3 and h != 5 and h != 6 and h != 7 and h != 8 ):
        # take square root if you must
        t = isqrt(n)
        if t*t == n:
            return t
        else:
            return -1
    
    return -1

## Wiener Attack (RSA1024)

Attacker only knows public parameters {n, e} and does not know any other parameters 

Goal: To check if implementation by user is careless and get the decryption exponent d to compromise encryption

In [84]:
def wiener_attack(e, n):
    frac = rational_to_contfrac(e, n)
    convergents = convergents_from_contfrac(frac)

    for (k ,d) in convergents:

        if k!=0 and (e*d-1) % k == 0:
            phi = (e*d-1) // k
            s = n - phi + 1

            discr = s * s - 4 * n

            if discr >= 0:
                t = is_perfect_square(discr)
                if t!= -1 and (s+t) % 2 == 0:
                    print("Hacked")
                    return d

In [85]:
times = 5

while times > 0:
    hacked_d = wiener_attack(e, N)

    if d == hacked_d:
        print('Hack Worked')
        hack_decrypted_message = RSA_decrypt(encrypted_message, hacked_d, N)
        print('Hacked decrypted message: ', convert_to_string(hack_decrypted_message))
        break
    else:
        print('Hack Failed')

    times -= 1

print('Initial message: ', convert_to_string(decrypted_message))

Hacked
Hack Worked
Hacked decrypted message:  Your Auntie
Initial message:  Your Auntie


## RSA 2048 

User of RSA encrypt message using RSA 1024

Generate primes p and q 

User deliberately generate a weak d exponent by setting max bit length to 1/3 * N^1/4 to get faster decryption 

Encryption exponent e (2048 bit) is obtained by inversing weak d 

In [86]:
phi_test = False

while phi_test == False: 
    p = generate_prime(1024, 3)
    q = generate_prime(1024, 3)
    
    phiN = (p-1) * (q-1)
    N = p * q

    d = RSA_generate_weakd(p, q)
    e = inv(d, phiN)

    if gcd(e, phiN) == 1:
        phi_test = True
    

    print('E is: ', e)
    print('E bit length: ', e.bit_length())

N is: 2338198001256711571927002181009324946646285878336959102881802876691669063918635280593974514182511905417504990071434609671133245083804383333049797396721237784302714137957662232182010092446101401070876604183435170487195928490669575940670153779059012435206859078627181597205553222690620244729890305599094895303117344534385914933361111069488926199828635429460671924669602118835401529045367605315965091272023816535464515186730734950074979514280603175556279418889622719276840494204143185729110515670713542450854479766565339641529295393807416729787796214176696301877295330809354831874778710380560677264785035142445321995163
Weak d =  1313880184814275290959196024837478095066791576938206739856571303099538047368014001480357876865224308510269773620412712514251810602247375019776774126132403
Weak d bit length:  509
E is:  1072878908932086446445027705663979976104432951122716078709090368797093162212125215977448560825207324561111926020477554050346014382869393654882874822052182088630234435717730587245382

In [87]:
message = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed"

ascii_message = convert_to_ascii(message)
ascii_message.bit_length()

encrypted_message = RSA_encrypt(ascii_message, e, N)
print('Encrypted message in ascii: ', encrypted_message)
print('Encrypted message in String: ', convert_to_string(encrypted_message))

Encrypted message in ascii:  2308017010787502998856045285768685994328318818504625998254833021094395836473181566923328829045646191925153017951207396996486717003705828735758447060998321215804482735990798132764041468363109278249795421845241058842225536217951096911309343952594398672602770997080582478776706403787959708767722308437889744560582846365122213997418734905538940723142196660805949620210879422672778544326809623468565098348797724634438018437535741734586339120335978322046042381205783643543084554316311759008526930223328446620461565281150860730479930551581494361358570994676029722472764113415691682421270490598883338271141046268715569704315
Encrypted message in String:  Ĵ
̓ǶϦ͘-ĝ̀ʭϢňľ̲ǸɱϦþ́^Ƌ̈́ǙµȶΛň̽-ʆ¿ΝηÏƌϤǦˍˁ̼˟˶ƿ<ϦŁ×̤Ǣ˟Ϟ̞˼)ǔūmĖừƥ͍ñ:͊áȘÙη`ΏĵŗθɒƎʠɚ̂ϥPɆǞ̈˂Ɠ̓ο˄˿˒ĴƵ͹˨ȰɆ͎ŭzÕϥƢ˞ΉȚά˓Äʔ̥εɬÒͯƦʠ̊Ƞņ̩ɯǔȵbŜ̝˔ɺƶƵȗ˥˞Ɋœxŏϒł.*ŽÍ̏ʃȟTȪļķȎ΢ßňƾɬǍȵę͜˚ǟ΢ȧɅǮũŦȺϢʤ˒ǘ˼qƟʳʪƥĎǪɖͳŒď.ČˋȹˀĻ


In [88]:
times = 5

while times > 0:
    hacked_d = wiener_attack(e, N)

    if d == hacked_d:
        print('Hack Worked')
        hack_decrypted_message = RSA_decrypt(encrypted_message, hacked_d, N)
        print('Hacked decrypted message: ', convert_to_string(hack_decrypted_message))
        break
    else:
        print('Hack Failed')

    times -= 1

decrypted_message = RSA_decrypt(encrypted_message, d, N)
print('Initial message: ', convert_to_string(decrypted_message))

Hacked
Hack Worked
Hacked decrypted message:  Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed
Initial message:  Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed
